The line (x_train, y_train), (x_test, y_test) = mnist.load_data() is responsible for loading the MNIST dataset using the mnist.load_data() function from the Keras library.

The MNIST dataset is a widely used benchmark dataset in machine learning and computer vision. It consists of a large number of grayscale images of handwritten digits (0-9), along with their corresponding labels.

When mnist.load_data() is called, it returns four NumPy arrays:

x_train: This array contains the training images. It is a 3-dimensional array of shape (num_train_samples, height, width), where num_train_samples represents the number of training samples, and height and width represent the dimensions of each image (28x28 pixels).
y_train: This array contains the labels for the training images. It is a 1-dimensional array of shape (num_train_samples,), where num_train_samples represents the number of training samples.
x_test: This array contains the testing images. It has the same structure as x_train, with dimensions (num_test_samples, height, width).
y_test: This array contains the labels for the testing images. It has the same structure as y_train, with dimensions (num_test_samples,).
By unpacking the values returned by mnist.load_data() into the variables (x_train, y_train), (x_test, y_test), the code assigns the training and testing images and labels to the corresponding variables for further processing and model training

Three-layer network on MNIST

In [16]:
!pip  install tensorflow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
!pip install keras

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import sys, numpy as np
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:1000].reshape(1000,28*28) / 255, y_train[0:1000]) # image and labels are two vectors basically.

one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))

for i,l in enumerate(y_test):
    test_labels[i][l] = 1

np.random.seed(1)
relu = lambda x: (x > 0) * x
relu2deriv = lambda x: (x > 0)

alpha, iterations, hidden_size, pixels_per_image, num_labels = (0.005, 350, 40, 784, 10)

weights_0_1 = 0.2 * np.random.random((pixels_per_image,hidden_size)) - 0.1
weights_1_2 = 0.2 * np.random.random((hidden_size,num_labels)) - 0.1

for j in range(iterations):
    error, correct_cnt = (0.0,0)
    for i in range(len(images)):
        layer_0 = images[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        layer_2 = np.dot(layer_1,weigths_1_2)
        
        error+= np.sum((labels[i:i+1] - layer_2) ** 2)  # layer_2 --> output and labels --> goal_prediction
        
        correct_cnt += int(np.argmax(layer_2) == np.argmax(labels)) # if both equal then returns 1 else return 0.
        
        layer_2_delta = labels[i:i+1] - layer_2
        layer_1_delta = layer_2_delta.dot(weigths_1_2.T) * relu2deriv(layer_1)
        
        # now we can change our weights so that we can make the error == 0.
        
        layer_2_weight_delta = layer_1.T.dot(layer_2_delta)
        layer_1_weight_delta = layer_0.T.dot(layer_1_delta)
        
        weights_1_2 += alpha * layer_2_weight_delta
        weights_0_1 += alpha * layer_1_weight_delta
        
        sys.stdout.write("\r" + "I " + str(j) + "error: " + str(error/float(len(images)))[0:5] + "corrections " + str(correct_cnt/float(len(images))))
        

I349error: 0.112corrections0.092